In [1]:
from modeling_esm_plusplus import ESMplusplusModel

In [3]:
model = ESMplusplusModel.from_pretrained('Synthyra/ESMplusplus_large')
tokenizer = model.tokenizer

sequences = ['MPRTEIN', 'MSEQWENCE']
tokenized = tokenizer(sequences, padding=True, return_tensors='pt')

# tokenized['labels'] = tokenized['input_ids'].clone() # correctly mask input_ids and set unmasked instances of labels to -100 for MLM training

output = model(**tokenized) # get all hidden states with output_hidden_states=True
print(output.last_hidden_state.shape) # last hidden state of the model, (batch_size, seq_len, hidden_size), (2, 11, 960)
#print(output.hidden_states) # all hidden states if you passed output_hidden_states=True (in tuple)

output = model(**tokenized, output_attentions=True)
att = output.attentions
len(att) # 30, one for each layer, size (batch_size, num_heads, seq_len, seq_len) each

torch.Size([2, 11, 1152])


36